In [1]:
from collections import defaultdict 
import pydot
import statistics
import os
import sys
import csv
import math
import random
import time
import FordFulkerson as ff
import EdmondsKarp as EK
from multiprocessing import Pool
from collections import Counter
from IPython.display import Image, display
from matplotlib import pyplot as plt
import numpy as np
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
path = "20%_test"
pathB = "/home/achilleas/Desktop/thesis/DATASET F1/specimen_benign/20%_test"
pathT = "80%_train"
paths = ["/home/achilleas/Desktop/thesis/DATASET F1/Fold_"+str(i) for i in range(1,6)]
# paths = "/home/achilleas/Desktop/thesis/DATASET F1/Fold_1"

filename = "default_G_tone_map"
mapFilename = "default_unique_mapping"

rootDirs = {}
trainDirs = {}

# rootDir = [os.path.join(root, name)
#          for root, dirs, files in os.walk(paths+"/"+path)
#          for name in files
#          if name.endswith(filename+".csv")]
# rootDir.sort()
# print(rootDir[0])

for i in range(len(paths)):
    p = [os.path.join(root, name)
             for root, dirs, files in os.walk(paths[i]+"/"+path)
             for name in files
             if name.endswith(filename+".csv")]
    p.sort()
    rootDirs[i] = p
# print(rootDirs[10])
benignDir = [os.path.join(root, name)
             for root, dirs, files in os.walk(pathB)
             for name in files
             if name.endswith(filename+".csv")]
benignDir.sort()

for i in range(len(paths)):
    trainDir = [os.path.join(root, name)
                 for root, dirs, files in os.walk(paths[i]+"/"+pathT)
                 for name in files
                 if name.endswith(filename+".csv")]
    trainDir.sort()
    trainDirs[i] = trainDir
    

In [3]:
classes = {"ACCESS_MASK":0,"Atom":1,"BOOLEAN":2,"Debug":3,"Device":4,
                                 "Environment":5,"File":6,"HANDLE":7,"Job":8,"LONG":9,"LPC":10,
                                 "Memory":11,"NTSTATUS":12,"Object":13,"Other":14,"PHANDLE":15,
                                 "PLARGE_INTEGER":16,"Process":17,"PUNICODE_STRING":18,
                                 "PULONG":19,"PULARGE_INTEGER":20,"PVOID_SIZEAFTER":21,
                                 "PWSTR":22,"Registry":23,"Security":24,"Synchronization":25,
                                 "Time":26,"Transaction":27,"ULONG":28,"WOW64":29, "DummyStart":30,"DummyEnd":31}

In [4]:
def getSourceSink(path):
    verteces = []
    with open(path) as fp:
        line = fp.readline()
        while line:
            verteces.append(line.split(","))
            line = fp.readline()
    return [verteces[0][0],verteces[-1][1].strip()]

In [5]:
def getArray (path):
    results = []
    with open(path) as csvfile:
        reader = csv.reader(csvfile,csv.QUOTE_NONNUMERIC) # change contents to floats
        for row in reader: # each row is a list
            nums = []
            for i in row: 
                if i :
                    nums.append(int(i))
            results.append(nums)
    return results

In [6]:
def createExtendedG(g):
    parents = []
    children = []
    for i in range(len(g)):
        for j in range(len(g[i])):
            if g[i][j]!=0:
                break
            else:
                children.append(i)
    for i in range(len(g)):
        for j in range(len(g[i])):
            if g[j][i]!=0:
                break
            else:
                parents.append(i)
                
    for i in range(len(g)):
        g[i].append(0)
        g[i].append(0)
    leng= len(g)
    g.append([0 for i in range (leng+2)])
    g.append([0 for i in range (leng+2)])
    
    for i in children :
        g[i][-1] = 1
    for j in parents:
        g[-2][j]=1
    return g

In [7]:
def createImage(g):
    G = pydot.Dot(graph_type='digraph')
    for i in range(len(g)):
        x = pydot.Node(i)
        for j in range(len(g[i])):
            if g[i][j]!= 0 :
                y = pydot.Node(j)
                e = pydot.Edge(i,j)
                G.add_edge(e)
                
    im = Image(G.create_png())
#     G.write_png(path)
    display(im)             

In [8]:
def findMaxOutDegreeVertex(g):
    outDegrees={}
    return_matrix = []
    for i in range(len(g)):
        for j in range(len(g[i])):
            if g[i][j]!=0:
                if i not in outDegrees:
                    outDegrees[i]=[g[i][j],1]
                else: 
                    weight = outDegrees[i][0]+g[i][j]
                    cardinality = outDegrees[i][1]+1
                    outDegrees[i]=[weight,cardinality]
    return outDegrees

In [9]:
def findMaxInDegreeVertex(g):
    inDegrees= {}
    return_matrix = []
    for i in range (len(g)):
        for j in range(len(g[i])):
            if g[j][i]!=0:
                if i not in inDegrees:
                    inDegrees[i] = [g[j][i],1]
                else: 
                    weight = inDegrees[i][0]+g[j][i]
                    cardinality = inDegrees[i][1]+1
                    inDegrees[i]=[weight,cardinality]
                    
    return inDegrees    

In [10]:
def createCoverageGraph(g):
    cvg = []
    combinedDegrees = {}
    inDegrees = findMaxInDegreeVertex(g)
    inKeys =list(inDegrees.keys())
    outDegrees = findMaxOutDegreeVertex(g)
    outKeys =list(outDegrees.keys())
    combinedDegrees= outDegrees.copy()
    for i in inKeys:
        if i in outDegrees:
            combinedDegrees[i] = [inDegrees[i][0]+outDegrees[i][0], inDegrees[i][1]+outDegrees[i][1]]
        else : 
            combinedDegrees[i] = inDegrees[i]
    sortedcDegrees = sorted(combinedDegrees.items(), key = lambda kv:kv[0])     
    cKeys = [i[0] for i in sortedcDegrees]
    for i in range(30):
        row = [0 for k in range(30)]
        if i not in cKeys:
            cvg.append(row)
            continue
        weight = combinedDegrees[i][0]
        cardinality = combinedDegrees[i][1]
        for j in cKeys:
            if weight > combinedDegrees[j][0] and cardinality > combinedDegrees[j][1]:
                row[j] = 1
        cvg.append(row)
        

    return cvg
    

# def writeCSV()
generic method to quickly write a graph to a csv file.

In [11]:
def writeCSV(g,path,name):
    path_spl = path.split('/')
    path_spl[-1] = name
    sp_path = '/'.join(path_spl)
    with open(sp_path, mode='w') as cvg_file:
        cvg_writer = csv.writer(cvg_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for i in g:
            cvg_writer.writerow(i)

# def executeTrial ()
## here we call the max Flow algorithm 
### Max flow in GRG
* if we get the source and the sink from the default G' the algorithm stops in cases of first caller X and last called X function.
* if we get the source and the sink from the default G_tone_cardinality tests show that nothing stops the algorithm as in all cases the first caller is different than the last called

### Max flow in CVG
* the algorithm works fine.


In [12]:
def executeTrial(path):
    sp_path = path.split('/')
    family = sp_path[-3]
    graph = getArray(path)
#     for i in range(len(graph)):
#         for j in range(len(graph[i])):
#             if graph[i][j]!=0:
#                 graph[i][j] =1
#     g = Graph(createExtendedG(createCoverageGraph(graph)))
    sp_path[-1]= "default_G_tone_cardinality.txt"
#     source = 30
#     sink = 31
    source = classes[getSourceSink('/'.join(sp_path))[0]]
    sink = classes[getSourceSink('/'.join(sp_path))[1]]
    fG = Graph(graph).FordFulkerson(source,sink)
    return [fG, family]

# def experiment()
wrapper method to execute and calculate median max flow for each member of our dataset
## median
we prefer median values of the flows because they are more robust to individual extreme flows

In [13]:
def experiment(dirs):
#     print(len(dirs))
    max_flow_vals={}
    mean_max_flow={}
    mean_flows = []
    total_values = []
    for i in dirs:
        values= executeTrial(i)
        total_values.append(values[0])
        if values[1] in max_flow_vals:
            max_flow_vals[values[1]].append(values[0])
        else : 
            max_flow_vals[values[1]]= [values[0]]

    for i in max_flow_vals:
        mean_max_flow[i] = statistics.median(max_flow_vals[i])
    for i in mean_max_flow :
        mean_flows.append(mean_max_flow[i])
        
    return mean_max_flow

# main body 
here we make separate calls in our functions to test the validity of the above methods

In [14]:
def executeExp():
    root_flows = experiment(rootDir)
    benign_flows = experiment(benignDir)
    train_flows = experiment(trainDir)
    # mal_corners = [min(root_flows), max(root_flows)]
    # benign_corners = [min(benign_flows), max(benign_flows)]


    # print("mal_corners:")
    # print(mal_corners)
    # print("benign_corners:")
    # print(benign_corners)

    sort_root = sorted(root_flows.items(), key = lambda kv:kv[1])
    sort_benign = sorted(benign_flows.items(), key = lambda kv:kv[1])
    sort_train = sorted(train_flows.items(), key = lambda kv:kv[1])

    # print()

    sort_root_vals = [i[1] for i in sort_root]
    sort_benign_vals = [i[1] for i in sort_benign]
    sort_train_vals = [i[1] for i in sort_train]


    sort_root_names = [i[0] for i in sort_root]
    sort_benign_names = [i[0] for i in sort_benign]
    sort_train_names = [i[0] for i in sort_train]


    sorted_root_flows = sorted(root_flows)
    sorted_benign_flows= sorted(benign_flows)



    # bCount =sorted(Counter(benign_flows).items(),key = lambda kv:kv[0])
    # mCount = sorted(Counter(root_flows).items(),key = lambda kv:kv[0])



    # --------------------------------------------
    lists = [sort_root_vals, sort_benign_vals,sort_train_vals]
    names = [sort_root_names, sort_benign_names, sort_train_names]

    for i in lists:
        plt.plot(i,marker=11)
    fig = plt.gcf()
    plt.show()
    plt.draw()

    fig.savefig('maxFlow.png')
    #---------------------------------------------------
    # executeTrial(rootDir[0])
    # ar = [[0,1,1,1,1],
    #       [0,0,0,1,1],
    #       [0,0,0,1,1],
    #       [0,0,0,0,1],
    #       [0,0,0,0,0]]
    # createExtendedG(ar)
    # cvg = createCoverageGraph(getArray(rootDir[0]),rootDir[0])

    # for i in rootDir:
    #     writeCSV(createExtendedG(createCoverageGraph(getArray(i))),i, 'CVGB.csv')
    # for i in cvg:
    #     print(i)
    # g = createExtendedG(ar)
    # G = pydot.Dot(graph_type='digraph')
    # for i in range(len(g)):
    #     x = pydot.Node(i)
    #     for j in range(len(g[i])):
    #         if g[i][j]!= 0 :
    #             y = pydot.Node(j)
    #             e = pydot.Edge(i,j)
    #             G.add_edge(e)

    # im = Image(G.create_png())
    # display(im)             

In [15]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

# Flow map
## def createFlowMap(g):
* Given the capacity of each edge and the neighbors of each node of a graph, the function returns a NxN array tha represents the flow map of the given graph, having in each `g'[i][j]= MaxFlow(cap,neighs,i,j)`. **the base function used to permutate our original GrGs** 

## def createMap(dirs):
* wrapper method that calls createFlowMap(path) for every path in the dirs list. This method is used to run through the rootDir, benignDir and train Dir to create our point of reference

## def CSM(A,B):
* calculates the cosine similarity metric for two arrays A and B
   

In [16]:
def createFlowMap(capacity, neighbors): 
    values = [[0 for i in range(len(capacity))] for j in range(len(capacity))]

    for i in range(len(capacity)):
        for j in range(len(capacity)):
            values[i][j] = EK.EdmondsKarp(capacity,neighbors,i,j)
#     return values
    return values

In [17]:
def createMap(paths):
    maps ={}
    median_vals = {}
    map_list = []
    test= {}
    for i in range(len(paths)):
        cap , neig = EK.ParseGraph(paths[i])
        test[i] = [cap, neig]
    
    for i in range(len(test)):
#         family = paths[i].split('/')[-3]
#         cap, neigh = EK.ParseGraph(paths[i])
        map_values = createFlowMap(test[i][0],test[i][1])
        map_list.append(np.array(map_values))
#         if family in maps:
#             maps[family].append(np.array(map_values))
#         else :
#             maps[family] =  [np.array(map_values)]
        printProgressBar(i, len(paths)-1, length = 50)       
    
    return map_list

In [18]:
def CSM(A,B):
    return 1-distance.cosine(A,B)

In [19]:
def checkDatabase(dirs, db, mode, iteration):
#     total = {}
#     for i in range(len(dirs)):
#         csm_val ={}
#         fname = dirs[i].split('/')[-3]
#         cap , neigh = EK.ParseGraph(dirs[i])
#         A = createFlowMap(cap,neigh)
#         for d in db:
#             val = []
#             for g in db[d]:
#                 val.append(CSM(A,g))
#             csm_val[d] = max(val)
#         printProgressBar(i,len(dirs))
#         sr = sorted(csm_val.items(), key = lambda kv:kv[1], reverse =True)
#         total[i]=[fname, sr[0][0], sr[0][1]]
# #         print([i, fname, sr[0][0], sr[0][1]])

    total = {}
    for i in range(len(dirs)):
        A = dirs[i]
        x,y = A.shape
        a = np.reshape(A,x*y)
        val = []
        f = 0
        for j in range(len(db)):
            B = db[j]
            w,u = B.shape
            b = np.reshape(B,w*u)
            val.append(CSM(a,b))
        trainIdx = val.index(max(val))
        if mode == 'malware':
            total[i] = [rootDirs[iteration][i].split('/')[-3],max(val),trainDirs[iteration][trainIdx].split('/')[-3]]
        if mode == 'benign':
            total[i] = [benignDir[i].split('/')[-3],max(val),trainDirs[iteration][trainIdx].split('/')[-3]]
        printProgressBar(i,len(dirs)-1, length =50)
#         print(i,rootDir[i].split('/')[-3],max(val),trainDir[val.index(max(val))].split('/')[-3])
    return total

In [20]:
# total = {}
#     for t in test:
#         for r in train:
#             csm_vals= {}
#             for i in range(len(test[t])):
#                 vals =[]
#                 for j in range(len(train[r])):
#                     vals.append(CSM(test[t][i],train[r][j]))
#                 csm_vals[t]=vals
#         total[t]= sorted(csm_vals[t], reverse = True)[0]

In [21]:
def similarityMetric(sr):
    counterA = 0
    counterB = 0
    counterC = 0
    for i in sr :
        A,B = i[0].split(',')[0], i[0].split(',')[1]
        C,D = i[2].split(',')[0], i[2].split(',')[1]
        if A==C and B==D:
            counterA +=1
        if A==C or B==D:
            counterB +=1
        if A==C or A==D or B == C or B == D:
            counterC += 1
    print("cA = %d (%f) cB= %d (%f) cC =%d (%f)"%(counterA, counterA/len(sr), counterB, counterB/len(sr), counterC, counterC/len(sr)))
    return [counterA/len(sr),counterB/len(sr),counterC/len(sr)]

In [22]:
def saveFigures(mal_th, ben_th, start, iteration):
    lists = [mal_th,ben_th]
    diffs = []
    v = 0
    for i in range(len(mal_th)):
        v = max(v,abs(mal_th[i]-ben_th[i]))
        diffs.append(abs(mal_th[i]-ben_th[i]))
    print(v)
    d = max(diffs)
    print(d, mal_th[diffs.index(d)])

    for i in lists:
        plt.plot(i,marker= 11)
    plt.title("max dif at %f"%(diffs[diffs.index(d)]))
    plt.legend(["True positives","False positives"])
    plt.yticks(np.arange(0, 1, 0.05))
    plt.xticks(np.arange(0,step, 1))
    plt.ylabel("")
    plt.draw()
    try:
        os.mkdir(str(iteration))
    except OSError:
#         print ("Creation of the directory %s failed" % str(iteration))
        pass
    else:
        print ("Successfully created the directory %s" % str(iteration))
    plt.savefig(str(iteration)+"/"+str(start)+".png")
    plt.clf()



In [23]:
database = []
for i in range(len(paths)):
    database.append(createMap(trainDirs[i]))
    print()
    print("%d database created"%(i))

 |██████████████████████████████████████████████████| 100.0% 

0 database created
 |██████████████████████████████████████████████████| 100.0% 

1 database created
 |██████████████████████████████████████████████████| 100.0% 

2 database created
 |██████████████████████████████████████████████████| 100.0% 

3 database created
 |██████████████████████████████████████████████████| 100.0% 

4 database created


In [24]:
benigns= createMap(benignDir)

 |██████████████████████████████████████████████████| 100.0% 


In [25]:
mals = {i: createMap(rootDirs[i]) for i in range(len(paths))}

 |██████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████| 100.0% 


In [26]:
malCheck = []
for i in range(len(paths)):
    ti = time.time()
    malCheck.append(checkDatabase(mals[i], database[i],'malware',i))
    print("\nmalcheck %d done."%(i))
    # with open('malwareCheck.txt', 'w') as f:
    #     for item in malwareCheck:
    #         f.write("%s\n" % malwareCheck[item])
    # f.close()
    print("check %d mal check finished in %f "%(i,time.time()-ti))
    ti = time.time()
    benignCheck = checkDatabase(benigns, database[i], 'benign',i)
    print("\nBencheck %d done."%(i)) 
    # with open('benignCheck.txt', 'w') as f:
    #     for item in benignCheck:
    #         f.write("%s\n" % benignCheck[item])
    # f.close()
    print("check %d ben check finished in %f s"%(i, time.time()-ti))
# malCheck = checkDatabase(mals,database[0],'malware', 0)

 |██████████████████████████████████████████████████| 100.0% 

malcheck 0 done.
check 0 mal check finished in 141.901745 
 |██████████████████████████████████████████████████| 100.0% 

Bencheck 0 done.
check 0 ben check finished in 9.105546 s
 |██████████████████████████████████████████████████| 100.0% 

malcheck 1 done.
check 1 mal check finished in 127.408297 
 |██████████████████████████████████████████████████| 100.0% 

Bencheck 1 done.
check 1 ben check finished in 8.972583 s
 |██████████████████████████████████████████████████| 100.0% 

malcheck 2 done.
check 2 mal check finished in 136.147456 
 |██████████████████████████████████████████████████| 100.0% 

Bencheck 2 done.
check 2 ben check finished in 8.570869 s
 |██████████████████████████████████████████████████| 100.0% 

malcheck 3 done.
check 3 mal check finished in 133.410905 
 |██████████████████████████████████████████████████| 100.0% 

Bencheck 3 done.
check 3 ben check finished in 9.468977 s
 |██████████████████████████

In [27]:
for mal_l in malCheck:
    val = []
    for j in mal_l:
        val.append(mal_l[j])
    val.sort(key= lambda kv : kv [1], reverse = True)
    similarityMetric(val)

cA = 346 (0.664107) cB= 420 (0.806142) cC =425 (0.815739)
cA = 346 (0.664107) cB= 419 (0.804223) cC =426 (0.817658)
cA = 343 (0.658349) cB= 415 (0.796545) cC =425 (0.815739)
cA = 340 (0.652591) cB= 412 (0.790787) cC =427 (0.819578)
cA = 332 (0.627599) cB= 414 (0.782609) cC =423 (0.799622)


In [28]:
mal_pairs = []
sr = []
avg = [[] for i in range(3)]
for i in range(len(malCheck)):
    mal_pairs.append([malCheck[i][j] for j in malCheck[i]])
    
for i in range(len(mal_pairs)):
    sr.append(sorted(mal_pairs[i],key =lambda kv:kv[-1],reverse=True))
    
ben_pairs = [benignCheck[i] for i in benignCheck]
br = sorted(ben_pairs,key =lambda kv:kv[-1],reverse=True)
sims = [similarityMetric(i) for i in sr]
for i in range(len(sims)):
    for j in range(len(sims[i])):
        avg[j].append(sims[i][j])
res = [statistics.mean(avg[i]) for i in range(len(avg)) ]
print(avg)
print("------------------")
print(res)

# for j in sr:
#     print(len(j))
#     for i in j:
#         print(i)
#     print("------------------")
# c =0
# # # m_vals =[sr[i][-1] for i in range(len(sr))]
# b_vals = [br[i][-1] for i in range(len(br))]
# for i in sr :
#     print(i)
# print("--------------------------")
# for j in br :
#     if j[-1] > 0.8059138664157343 :
#         c+=1
#     print(j)
# print(c)


cA = 346 (0.664107) cB= 420 (0.806142) cC =425 (0.815739)
cA = 346 (0.664107) cB= 419 (0.804223) cC =426 (0.817658)
cA = 343 (0.658349) cB= 415 (0.796545) cC =425 (0.815739)
cA = 340 (0.652591) cB= 412 (0.790787) cC =427 (0.819578)
cA = 332 (0.627599) cB= 414 (0.782609) cC =423 (0.799622)
[[0.6641074856046065, 0.6641074856046065, 0.6583493282149712, 0.6525911708253359, 0.6275992438563327], [0.8061420345489443, 0.8042226487523992, 0.7965451055662188, 0.7907869481765835, 0.782608695652174], [0.8157389635316699, 0.817658349328215, 0.8157389635316699, 0.8195777351247601, 0.7996219281663516]]
------------------
[0.6533509428211706, 0.796061086539264, 0.8136671879365333]


In [29]:
groupMal = {}
threshMal =[]
groupBen = {}
threshBen = []
for iteration in range(len(sr)):
    for j in br:
        start =j[1]
        step = 10
        dif = 1.0 - start
        thresh = [0 for i in range(step)]
        for i in range(step):
            thresh[i] = start + i*(dif/step)
        mal_th = [0 for i in range(len(thresh))]
        ben_th = [0 for i in range(len(thresh))]
        thresh_plots = []
        for i in range(len(thresh)):
            CM=0
            val=0
            for j in range(len(sr[iteration])):
                if sr[iteration][j][1] > thresh[i]:
                    #mal_th[i]+=1
                    CM += 1
            val=CM/521
            
            mal_th[i] = val
        for i in range(len(thresh)):
            CM=0
            val=0
            for j in range(len(br)):
                if br[j][1] > thresh[i]:
                    #mal_th[i]+=1
                    CM += 1
                
            val=CM/35
            ben_th[i] = val
        threshBen.append(ben_th)
        threshMal.append(mal_th)
        saveFigures(mal_th,ben_th,start,iteration)
    groupBen[iteration] = threshBen
    groupMal[iteration] = threshMal

0.5407183986838497
0.5407183986838497 0.9692898272552783
0.8332327940773239
0.8332327940773239 0.8618042226487524
0.8276939950644365
0.8276939950644365 0.8848368522072937
0.78206745270085
0.78206745270085 0.8963531669865643
0.8255552508911433
0.8255552508911433 0.8541266794625719
0.8505072662462297
0.8505072662462297 0.8790786948176583
0.8447491088565945
0.8447491088565945 0.8733205374280231
0.8409103372635043
0.8409103372635043 0.8694817658349329
0.6605429119824513
0.6605429119824513 0.946257197696737
0.6605429119824513
0.6605429119824513 0.946257197696737
0.6605429119824513
0.6605429119824513 0.946257197696737
0.2970112421168083
0.2970112421168083 0.982725527831094
0.8332327940773239
0.8332327940773239 0.8618042226487524
0.7078694817658349
0.7078694817658349 0.9078694817658349
0.8274746366876885
0.8274746366876885 0.8560460652591171
0.8428297230600493
0.8428297230600493 0.8714011516314779
0.8447491088565945
0.8447491088565945 0.8733205374280231
0.8029613380860982
0.8029613380860982 0

0.8773786673978613
0.8773786673978613 0.9059500959692899
0.8716205100082259
0.8716205100082259 0.9001919385796545
0.8658623526185906
0.8658623526185906 0.8944337811900192
0.6758979983548122
0.6758979983548122 0.9616122840690979
0.6758979983548122
0.6758979983548122 0.9616122840690979
0.6739786125582671
0.6739786125582671 0.9596928982725528
0.31620510008225944
0.31620510008225944 1.0019193857965452
0.8658623526185906
0.8658623526185906 0.8944337811900192
0.7366602687140116
0.7366602687140116 0.9366602687140115
0.8620235810255004
0.8620235810255004 0.8905950095969289
0.8658623526185906
0.8658623526185906 0.8944337811900192
0.8697011242116808
0.8697011242116808 0.8982725527831094
0.8279133534411846
0.8279133534411846 0.9136276391554703
0.7462571976967369
0.7462571976967369 0.946257197696737
0.7289827255278312
0.7289827255278312 0.9289827255278311
0.8620235810255004
0.8620235810255004 0.8905950095969289
0.8562654236358651
0.8562654236358651 0.8848368522072937
0.8618042226487524
0.861804222

<Figure size 432x288 with 0 Axes>